In [2]:
from sklearn import datasets
from datasets import load_dataset

# Load CNN/DailyMail (phiên bản non-anonymized, thường dùng)
dataset = load_dataset("cnn_dailymail", "3.0.0")

print(dataset)
print(dataset["train"][0])


ModuleNotFoundError: No module named 'datasets'

In [ ]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

max_input_len = 512
max_target_len = 128

def preprocess(batch):
    inputs = tokenizer(batch["article"], max_length=max_input_len, truncation=True, padding="max_length")
    targets = tokenizer(batch["highlights"], max_length=max_target_len, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

encoded_dataset = dataset.map(preprocess, batched=True, remove_columns=["article","highlights","id"])
encoded_dataset.set_format(type="torch")


In [ ]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,   # nhỏ vì Colab RAM ít
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"].select(range(5000)),  # train nhỏ cho nhanh
    eval_dataset=encoded_dataset["validation"].select(range(500)),
)

trainer.train()


In [ ]:
test_text = dataset["test"][0]["article"]
inputs = tokenizer([test_text], max_length=512, truncation=True, return_tensors="pt")
summary_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)

print("Original:", test_text[:500], "...")
print("Gold Summary:", dataset["test"][0]["highlights"])
print("Pred Summary:", tokenizer.decode(summary_ids[0], skip_special_tokens=True))
